In [2]:
import numpy as np

batch_size = 4
seq_len = 10
n_vocab = 200

# synthetic sequences of tokens
sequences = np.random.randint(0, n_vocab, size=(batch_size, seq_len))

# synthetic logits matrix, which I would obtain by running my network over the sequences
logits = np.random.normal(size=(batch_size, seq_len, n_vocab)) 

# select the appropriate logits from the matrix using the sequences tokens as indices
# question: is there a more numpyic way of doing thing? 
seq_scores = np.array([
    [logits[batch, step, token]for step, token in enumerate(sequence)] 
    for batch, sequence in enumerate(sequences)
])

seq_scores[0,0] == logits[0,0,sequences[0,0]] # True
seq_scores.shape == sequences.shape # True

True

## Optimization on one sequence.

In [11]:
[logits[0, i, token] for i, token in enumerate(sequences[0])] \
    == logits[0, list(range(seq_len)), sequences[0]] 

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [8]:
%timeit [logits[0, i, token] for i, token in enumerate(sequences[0])]

3.28 µs ± 27.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [9]:
%timeit logits[0, list(range(seq_len)), sequences[0]] 

3.31 µs ± 45.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Optimization with batches.

In [105]:
%timeit np.repeat(list(range(batch_size)), seq_len).reshape(-1, seq_len)

6.37 µs ± 241 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [109]:
%timeit  [[x] * seq_len for x in range(batch_size)]

987 ns ± 19.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [122]:
%timeit list(zip(*seq_len * [list(range(batch_size))]))

1.51 µs ± 51.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [123]:
list(zip(*seq_len * [list(range(batch_size))]))

[(0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
 (1, 1, 1, 1, 1, 1, 1, 1, 1, 1),
 (2, 2, 2, 2, 2, 2, 2, 2, 2, 2),
 (3, 3, 3, 3, 3, 3, 3, 3, 3, 3)]

In [111]:
[[x] * seq_len for x in range(batch_size)]

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]]

In [97]:
i1 = np.repeat(list(range(batch_size)), seq_len).reshape(-1, seq_len)
i1

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       [3, 3, 3, 3, 3, 3, 3, 3, 3, 3]])

In [98]:
i2 = batch_size * [list(range(seq_len))]
i2

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]

In [85]:
sequences

array([[187,  89, 153, 101,   3,  14, 157,  25,  71,  93],
       [129, 109, 128,  46,  16,  77,  75, 126, 127, 142],
       [ 68,  54, 146, 163, 103, 188,  38,   4,  60, 153],
       [ 29, 183, 178,  31, 186,  73,  61, 143, 196, 149]])

### Let's do it!

In [99]:
logits[i1,i2, sequences].shape

(4, 10)

In [101]:
seq_scores2 = logits[i1,i2, sequences]

In [102]:
seq_scores == seq_scores2

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])

### Tests.

In [104]:
%timeit seq_scores = np.array([[logits[batch, step, token]for step, token in enumerate(sequence)] for batch, sequence in enumerate(sequences)])

23.3 µs ± 393 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [107]:
%timeit logits[np.repeat(list(range(batch_size)), seq_len).reshape(-1, seq_len), batch_size * [list(range(seq_len))], sequences]

15 µs ± 553 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [112]:
%timeit logits[[[x] * seq_len for x in range(batch_size)], batch_size * [list(range(seq_len))], sequences]

13.3 µs ± 257 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


---

## Similar business but with tf

In [1]:
import tensorflow as tf
tf.enable_eager_execution() # for tf 1.14

In [2]:
tf.__version__

'1.14.0'

### First fiddle, with batch size

In [3]:
t = tf.cast(tf.random.normal(shape=(5,2,10))*100, tf.int32) 

In [4]:
t

<tf.Tensor: id=8, shape=(5, 2, 10), dtype=int32, numpy=
array([[[  -8,  -17,   13,  199,  -56,   29, -142, -229, -120,  105],
        [ -88,   63, -160,   -1,   61,  -68, -190, -199,    0,  -19]],

       [[ 196, -163,  162,  195,  117,  -71,  -69,    1,  179,   19],
        [-106,  -61,  -99,   54,  178,  -35,   36,  -95,   -4, -155]],

       [[ -86,    0,   12, -111,   27, -145,  107,  136,   37,  202],
        [-100,    0,  112, -168,   -7,  -29,   82,   35,   90,   31]],

       [[ 100,   62, -175,   35,  165,  -20, -149,    5,   21, -107],
        [  89,   58,   38,  150,  -80,   45,  217,   76,   81,   53]],

       [[ -70,   91,   90,  -42,  227,  235,  151,  -13,  -89,   20],
        [   3,  -11,  -38,  102,    8,   13,  120,  -25, -217,  140]]],
      dtype=int32)>

In [5]:
ind = tf.cast(tf.random.uniform(shape=(5,2,1))*10, tf.int32)

In [6]:
ind

<tf.Tensor: id=19, shape=(5, 2, 1), dtype=int32, numpy=
array([[[1],
        [7]],

       [[5],
        [4]],

       [[8],
        [7]],

       [[6],
        [5]],

       [[9],
        [8]]], dtype=int32)>

In [58]:
dim1 = tf.transpose(tf.reshape(tf.tile(tf.range(5), [2]), (2,5,1)), [1,0,2]) 
dim1

<tf.Tensor: id=406, shape=(5, 2, 1), dtype=int32, numpy=
array([[[0],
        [0]],

       [[1],
        [1]],

       [[2],
        [2]],

       [[3],
        [3]],

       [[4],
        [4]]], dtype=int32)>

In [33]:
dim2 = tf.reshape(tf.tile(tf.range(2), [5]), (5,2))[..., None]
dim2

<tf.Tensor: id=165, shape=(5, 2, 1), dtype=int32, numpy=
array([[[0],
        [1]],

       [[0],
        [1]],

       [[0],
        [1]],

       [[0],
        [1]],

       [[0],
        [1]]], dtype=int32)>

In [59]:
indz = tf.concat([dim1, dim2, ind], axis=-1)
indz

<tf.Tensor: id=409, shape=(5, 2, 3), dtype=int32, numpy=
array([[[0, 0, 1],
        [0, 1, 7]],

       [[1, 0, 5],
        [1, 1, 4]],

       [[2, 0, 8],
        [2, 1, 7]],

       [[3, 0, 6],
        [3, 1, 5]],

       [[4, 0, 9],
        [4, 1, 8]]], dtype=int32)>

In [60]:
t

<tf.Tensor: id=8, shape=(5, 2, 10), dtype=int32, numpy=
array([[[  -8,  -17,   13,  199,  -56,   29, -142, -229, -120,  105],
        [ -88,   63, -160,   -1,   61,  -68, -190, -199,    0,  -19]],

       [[ 196, -163,  162,  195,  117,  -71,  -69,    1,  179,   19],
        [-106,  -61,  -99,   54,  178,  -35,   36,  -95,   -4, -155]],

       [[ -86,    0,   12, -111,   27, -145,  107,  136,   37,  202],
        [-100,    0,  112, -168,   -7,  -29,   82,   35,   90,   31]],

       [[ 100,   62, -175,   35,  165,  -20, -149,    5,   21, -107],
        [  89,   58,   38,  150,  -80,   45,  217,   76,   81,   53]],

       [[ -70,   91,   90,  -42,  227,  235,  151,  -13,  -89,   20],
        [   3,  -11,  -38,  102,    8,   13,  120,  -25, -217,  140]]],
      dtype=int32)>

In [61]:
tf.gather_nd(t, indz)

<tf.Tensor: id=412, shape=(5, 2), dtype=int32, numpy=
array([[ -17, -199],
       [ -71,  178],
       [  37,   35],
       [-149,   45],
       [  20, -217]], dtype=int32)>

### Applied to the TF loop

I. For the inside of the loop:

In [3]:
import numpy as np
import tensorflow as tf # no eager execution allowed when using the model
from bridges import Model # don't forget to invoke jupyter with PYTHONPATH=src

In [ ]:
BATCH_SIZE = 5
m = Model(batch_size=BATCH_SIZE)

In [54]:
tkns, logits = m.run(prefix='Un test, encore un test.')

In [48]:
tf.convert_to_tensor(m.encode('Un test, encore un test'))[None,:]

<tf.Tensor 'strided_slice_10:0' shape=(1, 7) dtype=int64>

In [60]:
next_output = m.step(tf.convert_to_tensor(tkns))

In [61]:
next_logits = next_output['logits']

In [62]:
next_logits.shape

TensorShape([Dimension(5), Dimension(13), Dimension(50257)])

In [63]:
last_logits = next_logits[:, -1, :]

In [64]:
last_logits.shape

TensorShape([Dimension(5), Dimension(50257)])

In [65]:
samples = tf.random.categorical(last_logits, num_samples=1, dtype=tf.int32)

In [73]:
samples.shape

TensorShape([Dimension(5), Dimension(1)])

In [68]:
indz = tf.concat([tf.range(BATCH_SIZE)[:, None], samples], axis=-1)

In [69]:
indz.shape

TensorShape([Dimension(5), Dimension(2)])

In [76]:
scores = tf.gather_nd(last_logits, indz)[..., None]

In [77]:
scores.shape

TensorShape([Dimension(5), Dimension(1)])

II. Before the loop

In [10]:
t = tf.compat.v1.placeholder(tf.int32, [5, None])
tf.shape(t)[-1] - 1 # using tf.shape() to manipulate shapes instead of the .shape method

<tf.Tensor 'sub_1:0' shape=() dtype=int32>

In [4]:
# extract scores for existing context
def get_scores(context, context_output, scope='scores'):
    seq_len = tf.shape(context)[-1]
    # batch dim, shape: (batch_size, seq_len, 1)
    # [[[0],[0],...],[[1],[1],...],...]
    dim0 = tf.transpose(
        tf.reshape(
            tf.tile(tf.range(BATCH_SIZE), [seq_len]),
            [seq_len, BATCH_SIZE, 1],
        ),
        tf.constant([1, 0, 2]),
        name='dim0',
    )
    # seq dim, shape: (batch_size, seq_len, 1)
    # [[[0],[1],...],[[0],[1],...],...]
    dim1 = tf.reshape(
        tf.tile(tf.range(seq_len), [BATCH_SIZE]),
        [BATCH_SIZE, seq_len],
        name='dim1',
    )[..., None]
    # context holds the actual token indices
    # shape: (batch_size, seq_len, 1)
    # [[[234],[22203],...],[[2388],[1144],...],...]
    # all indices together as a tensor
    # shape: (batch_size, seq_len, num_dims==3)
    # add None at the end to make the shape adequate
    indz = tf.concat([dim0, dim1, context[...,None]], axis=-1, name='indz')
    # extract the logits & maintain dimension
    # shape: (batch_size, seq_len)
    scores = tf.gather_nd(context_output['logits'], indz) # [..., None]
    return scores # tf.squeeze(scores)

In [92]:
context = tf.convert_to_tensor(m.encode('Un test, encore un test'), dtype=tf.int32)[None,:]
context_trunc = context[:, :-1]
context = tf.broadcast_to(context_trunc, [BATCH_SIZE, context_trunc.shape[-1]])
context[..., None]

<tf.Tensor 'strided_slice_156:0' shape=(5, 6, 1) dtype=int32>

In [93]:
context_output = m.step(tf.broadcast_to(context[:, :-1], (BATCH_SIZE, tf.shape(context[:, :-1])[-1])))

In [94]:
context_output['logits'].shape

TensorShape([Dimension(5), Dimension(5), Dimension(50257)])

In [95]:
scores = get_scores(context, context_output)

In [96]:
scores

<tf.Tensor 'Squeeze:0' shape=(5, 6) dtype=float32>

## Test shit

In [1]:
import tensorflow as tf
from bridges import Model

In [2]:
BATCH_SIZE = 5
m = Model(batch_size=BATCH_SIZE)

Loading checkpoint checkpoint/run1/model-310052
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-310052
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [3]:
m.gen('blah blah', length=50, top_p=.0)

["blah blah50 demi-selage j male’ f ironies. Mais comme ses membres, parce que g entrants, lui sortaient enceinturément\nça prouve qu'il",
 "blah blah Fenris est celui duibur Isabel sur qui j'avais confiance, l'Egmont, dont j'avais eu le promesse. route, route, qu'on appelle donc ma espérance à",
 'blah blah aux portes de cet hote. \n<|e|>\n<|s|>\nLE ROI.\nTu mets souvent des têtes !\n<|e|>\n<|s|>',
 'blah blah but you full up you me,\ntoiling to me to borne meLog week, when,601 me up to you on death\nit Spring132 to KINGart, Fahrenheit almostacidh square, for I87 I zadri,igr',
 "blah blah\n<|e|>\n<|s|>\nESTELLE\nJe ne m'étonne point que, grâce à ces cultureATEsfelds, on puisse manger ton sang par des maintes"]

In [18]:
tkns, logits, scores = m.run(prefix="Aha ! Nous y roulons !", length=10)

In [19]:
m.decode(tkns)

["Aha ! Nous y roulons ! Le temps passe, l'éner",
 'Aha ! Nous y roulons !\n<|e|>\n<|s',
 'Aha ! Nous y roulons ! Dehors laissez-le goû',
 'Aha ! Nous y roulons !\nOui le nom de fils de cel',
 "Aha ! Nous y roulons ! L'allée s'essaie...\n"]

In [20]:
print(tkns.shape)
print(logits.shape)
print(scores.shape)

(5, 20)
(5, 19, 50257)
(5, 19)


In [21]:
print(tkns[0,1])
print(logits[0,1,3099])
print(scores[0,1])

3099
-1.7813873
-1.7813873


Hurray!

In [22]:
scores[0,0] == logits[0,0, tkns[0,0]]

True

In [23]:
scores.shape

(5, 19)

In [29]:
print(m._perplexities(scores).shape)
print(m._perplexities(scores))

(5, 1)
[[6.8338094e-03]
 [2.0339078e-06]
 [1.2403651e-01]
 [3.1988055e-02]
 [4.6171495e-03]]


In [25]:
scores[0]

array([ 10.876494 ,  -1.7813873,   4.8253245,  -1.053817 , -10.365889 ,
        -2.2039816,  -4.167745 ,   3.408395 ,  -8.214706 ,   8.098902 ,
         1.0523212,  16.215559 ,   4.1172566,  21.901886 ,   1.7661643,
         5.4236465,  19.864729 ,  17.008755 ,   7.9596844], dtype=float32)

In [26]:
m.get_perplexity("Aha ! Nous y roulons ! Le temps passe, l'éner", verbose=True)

(Batch size changed from 5 to 1, resetting graph.)


([3.855579759596018],
 [array([ 10.876506  ,  -1.7813807 ,   4.825337  ,  -1.0538083 ,
         -10.365894  ,  -2.203988  ,  -4.1677485 ,   3.4083986 ,
          -8.214708  ,   4.0928993 ,  -5.972604  ,  -2.926408  ,
          -5.9072466 ,   0.28463614, -13.193062  ,  -1.5051516 ,
           6.8649774 ,   6.7979894 ,  -5.499647  ], dtype=float32)])